In [156]:
import numpy as np

In [213]:
def data_iter(x, y, batch_size):
    num=len(x)
    indices=list(range(num))
    np.random.shuffle(indices)
    for i in range(0,num,batch_size):
        batch_indices=np.array(indices[i:i+batch_size])
        yield x[batch_indices],y[batch_indices]

def relu(x):
    return np.maximum(x,0)
def sigmoid(x):
    return 1/(1+np.exp(-x))
def MSE(y1,y2):
    return np.sum((y1-y2)**2)/2



In [158]:
import torch
w=torch.tensor([[3],[2],[4.0]],dtype=torch.float,requires_grad=True)
x=torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float,requires_grad=True)


In [159]:
h=x@w
h.sum().backward()

w.grad,x,h,w


(tensor([[5.],
         [7.],
         [9.]]),
 tensor([[1., 2., 3.],
         [4., 5., 6.]], requires_grad=True),
 tensor([[19.],
         [46.]], grad_fn=<MmBackward0>),
 tensor([[3.],
         [2.],
         [4.]], requires_grad=True))

In [350]:
true_w1=np.array([[2,4,3,4],[4,5,6,5],[6,7,8,9]])
true_w2=np.array([[2,4,3,4]]).reshape(4,1)
true_b1=np.array([[3,4,5,6]])
true_b2=np.array([2])

x=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]])
y=x@true_w1+true_b1
y=relu(y)
y=y@true_w2+true_b2
y

array([[ 540],
       [1173],
       [1806],
       [2439],
       [3072]])

In [351]:
from torch import nn
net=nn.Sequential(nn.Linear(3,4),nn.Sigmoid(),nn.Linear(4,1))
net.zero_grad()
net[0].weight.data.normal_(0,0.01)
# net[0].bias.data.zero_()
net[0].bias.data.fill_(0)
net[2].weight.data.normal_(0,0.01)
net[2].bias.data.fill_(0)

# net[2].weght,net[2].bias=nn.Parameter(w2_tensor,requires_grad=True),nn.Parameter(b2_tensor,requires_grad=True)
x_data=torch.tensor(x,dtype=torch.float)


y_data=torch.tensor(y,dtype=torch.float)

loss=nn.MSELoss()(net(x_data),y_data)
loss.backward(torch.ones_like(loss))
net(x_data)

tensor([[-0.0234],
        [-0.0235],
        [-0.0235],
        [-0.0235],
        [-0.0235]], grad_fn=<AddmmBackward0>)

In [352]:
w1=net[0].weight.detach().numpy().T
b1=net[0].bias.detach().numpy()
w2=net[2].weight.detach().numpy().T
b2=net[2].bias.detach().numpy()
def forward(activate):
    global H2,H1
    global y_hat
    H1=x@w1+b1
    if activate=='relu':
        H2=relu(H1)
    elif activate=='sigmoid':
        H2=sigmoid(H1)
    y_hat=H2@w2+b2
    
    return y_hat
w1,w2

(array([[ 0.00224926,  0.00796723, -0.00811446,  0.01140718],
        [ 0.01547366, -0.00307367, -0.01126522, -0.00485089],
        [ 0.00265518, -0.004004  ,  0.00885585,  0.02075562]],
       dtype=float32),
 array([[-0.00797619],
        [-0.02229787],
        [-0.01568101],
        [-0.00084921]], dtype=float32))

In [353]:
def grad_cal(activation):
    dloss_y=2*(y_hat-y)
    dloss_w2=np.dot(H2.T,dloss_y)/(y.shape[0]*y.shape[1])
    dloss_b2=dloss_y.sum(axis=0)/(y.shape[0]*y.shape[1])
    
    dloss_H2=np.dot(dloss_y,w2.T)
    dH2_H1=np.zeros(dloss_H2.shape)
    if activation=='relu':
        dH2_H1[H2>0]=1
    elif activation=='sigmoid':
        dH2_H1=sigmoid(H1)*(1-sigmoid(H1))
    dloss_H1=dloss_H2*dH2_H1
    dloss_w1=np.dot(x.T,dloss_H1)/(y.shape[0]*y.shape[1])
    dloss_b1=dloss_H1.sum(axis=0)/(y.shape[0]*y.shape[1])
    return dloss_w1,dloss_b1,dloss_w2,dloss_b2



In [354]:
net[0].weight.grad,net[0].bias.grad,net[2].weight.grad,net[2].bias.grad

(tensor([[ 64.6035,  71.7202,  78.8370],
         [183.2899, 203.4250, 223.5602],
         [128.5160, 142.6427, 156.7693],
         [  6.7888,   7.5385,   8.2881]]),
 tensor([ 7.1167, 20.1351, 14.1266,  0.7496]),
 tensor([[-1991.4038, -1803.3289, -1725.4192, -2061.3118]]),
 tensor([-3612.0469]))

In [355]:
forward(activate='sigmoid')
grad_cal('sigmoid')

(array([[ 64.60348289, 183.28989828, 128.51601873,   6.7888304 ],
        [ 71.72022629, 203.42503646, 142.64264771,   7.53847806],
        [ 78.83696968, 223.56017464, 156.7692767 ,   8.28812571]]),
 array([ 7.1167434 , 20.13513818, 14.12662898,  0.74964766]),
 array([[-1991.40388268],
        [-1803.32884463],
        [-1725.41918593],
        [-2061.31185416]]),
 array([-3612.04696871]))

In [394]:
class net_2:
    def __init__(self, inputs, outputs, num_1,batch_size=1 , loss_fun='MSE', activate='relu'):
        self.dloss_b1 = None
        self.dloss_w1 = None
        self.dloss_b2 = None
        self.dloss_w2 = None
        self.H2 = None
        self.H1 = None
        self.y_hat=None
        self.batch_size =batch_size
        
        self.inputs=inputs
        self.outputs=outputs
        self.num_1=num_1
        self.loss=loss_fun
        self.activate=activate

        
        self.w1=np.random.normal(0,0.01,(inputs,num_1))
        self.b1=np.zeros((1,num_1))
        self.w2=np.random.normal(0,0.01,(num_1,outputs))
        self.b2=np.zeros((1,outputs))
    @property 
    def weights(self):
        return self.w1,self.w2
    @property
    def bias(self):
        return self.b1,self.b2
    def forward(self, x_datas):

        self.H1= x_datas @ self.w1 + self.b1
        if self.activate=='relu':
            self.H2=relu(self.H1)
        elif self.activate=='sigmoid':
            self.H2=sigmoid(self.H1)
        self.y_hat=self.H2@self.w2+self.b2
        
        return self.y_hat

    def backward(self, x_datas, y_datas):
        dloss_y=1
        if self.loss=='MSE':
            dloss_y=2*(self.y_hat - y_datas)
        self.dloss_w2=np.dot(self.H2.T,dloss_y)/(self.y_hat.shape[0]*self.y_hat.shape[1])
        self.dloss_b2=dloss_y.sum(axis=0)/(self.y_hat.shape[0]*self.y_hat.shape[1])
        
        dloss_H2=np.dot(dloss_y,self.w2.T)
        dH2_H1=np.zeros(dloss_H2.shape)
        if self.activate=='relu':
            dH2_H1[self.H2>0]=1
        elif self.activate=='sigmoid':
            dH2_H1=sigmoid(self.H1)*(1-sigmoid(self.H1))
        dloss_H1=dloss_H2*dH2_H1
        self.dloss_w1= np.dot(x_datas.T, dloss_H1) / (self.y_hat.shape[0] * self.y_hat.shape[1])
        self.dloss_b1=dloss_H1.sum(axis=0)/(self.y_hat.shape[0]*self.y_hat.shape[1])
        return self.dloss_w1,self.dloss_b1,self.dloss_w2,self.dloss_b2
    
    def updater(self,lr=0.0001):
        #update

        self.w1-=lr*self.dloss_w1/self.batch_size
        self.b1-=lr*self.dloss_b1/self.batch_size
        self.w2-=lr*self.dloss_w2/self.batch_size
        self.b2-=lr*self.dloss_b2/self.batch_size
net_re=net_2(3,1,4,batch_size=5 ,loss_fun='MSE',activate='relu')

for i in range(50):
    net_re.forward(x)
    net_re.backward(x,y)
    net_re.updater()
    if i%10==0:
        print(net_re.weights)

(array([[-0.01375758, -0.00131167, -0.01771351, -0.00930163],
       [-0.0040747 , -0.00506878,  0.01660477, -0.00912746],
       [-0.00593675,  0.00190967, -0.00326815, -0.02148413]]), array([[-0.00463227],
       [ 0.00251828],
       [-0.01060505],
       [-0.00676628]]))
(array([[-0.01375758, -0.00131167, -0.01812284, -0.00930163],
       [-0.0040747 , -0.00506878,  0.0157861 , -0.00912746],
       [-0.00593675,  0.00190967, -0.00449615, -0.02148413]]), array([[-0.00463227],
       [ 0.00251828],
       [-0.01049196],
       [-0.00676628]]))
(array([[-0.01375758, -0.00131167, -0.01812284, -0.00930163],
       [-0.0040747 , -0.00506878,  0.0157861 , -0.00912746],
       [-0.00593675,  0.00190967, -0.00449615, -0.02148413]]), array([[-0.00463227],
       [ 0.00251828],
       [-0.01049196],
       [-0.00676628]]))
(array([[-0.01375758, -0.00131167, -0.01812284, -0.00930163],
       [-0.0040747 , -0.00506878,  0.0157861 , -0.00912746],
       [-0.00593675,  0.00190967, -0.00449615, -0

In [381]:
true_w1,true_w2

(array([[2, 4, 3, 4],
        [4, 5, 6, 5],
        [6, 7, 8, 9]]),
 array([[2],
        [4],
        [3],
        [4]]))